In [1]:
import pandas as pd
from stock_screening_local import *
import pickle

In [2]:
ticker_info = pd.read_csv('yahoo_tickers.csv')
all_tickers = ticker_info['Ticker'].tolist()
all_tickers.sort()

In [21]:
ticker_ratios = {ticker: {} for ticker in all_tickers}
for i, ticker in enumerate(all_tickers):
    print(i, ticker)
    ticker_ratios[ticker] = {'quick_ratio': quick_ratio(ticker, latest=True, period=-1),
                             'cash_ratio': cash_ratio(ticker, latest=True, period=-1),
                             'operating_cash_ratio': operating_cash_ratio(ticker, latest=True, period=-1),
                             'interest_coverage_ratio': interest_coverage_ratio(ticker, latest=True, period=-1),
                             'debt_service_coverage_ratio': debt_service_coverage_ratio(ticker, latest=True, period=-1),
                             'asset_coverage_ratio': asset_coverage_ratio(ticker, latest=True, period=-1),
                             'cash_coverage_ratio': cash_coverage_ratio(ticker, latest=True, period=-1),
                             'total_debt_to_tangible_book_value_ratio': total_debt_to_tangible_book_value_ratio(ticker, latest=True, period=-1),
                             'total_debt_to_total_assets_ratio': total_debt_to_total_assets_ratio(ticker, latest=True, period=-1),
                             'total_debt_to_EBITDA_ratio': total_debt_to_EBITDA_ratio(ticker, latest=True, period=-1)}
    if all([v is not None for v in ticker_ratios[ticker].values()]):
        print(f'{i} {ticker} PASSED!')
with open('ticker_ratios.pickle', 'wb') as f:
    pickle.dump(ticker_ratios, f)
with open('output.txt', 'r') as f:
    lines = f.read().splitlines()
missing_fields = {}
for line in lines:
    field = line.split(': ')[-1]
    if field.startswith('\'') and field.endswith('\'') and 'NoneType' not in field:
        try:
            missing_fields[field] += 1
        except KeyError:
            missing_fields[field] = 1
missing_fields = {k: missing_fields[k] for k in sorted(missing_fields.keys(), key=missing_fields.get, reverse=True)}
missing_fields

0 000002.SZ
'annualEBITDA'
'annualDepreciationAmortizationDepletion'
'annualAmortizationOfSecurities'
'annualEBITDA'
'annualDepreciationAmortizationDepletion'
'annualAmortizationOfSecurities'
0 000002.SZ PASSED!
1 000333.SZ
'annualEBITDA'
'annualDepreciationAmortizationDepletion'
'annualReconciledDepreciation'
'annualReconciledDepreciation'
'annualAmortizationOfSecurities'
unsupported operand type(s) for +: 'float' and 'NoneType'
'annualReconciledDepreciation'
'annualDepreciationAmortizationDepletion'
'annualEBITDA'
'annualDepreciationAmortizationDepletion'
'annualReconciledDepreciation'
'annualReconciledDepreciation'
'annualAmortizationOfSecurities'
unsupported operand type(s) for +: 'float' and 'NoneType'
2 000858.SZ
'annualEBITDA'
'annualDepreciationAmortizationDepletion'
'annualAmortizationOfSecurities'
'annualCurrentDebt'
'annualLongTermDebt'
'annualEBITDA'
'annualDepreciationAmortizationDepletion'
'annualAmortizationOfSecurities'
3 002142.SZ
'annualCurrentAssets'
'annualTotalNonC

{"'annualEBITDA'": 1010,
 "'annualDepreciationAmortizationDepletion'": 321,
 "'annualAmortizationOfSecurities'": 308,
 "'annualCurrentLiabilities'": 163,
 "'annualTotalNonCurrentLiabilitiesNetMinorityInterest'": 163,
 "'annualInventory'": 130,
 "'annualEBIT'": 123,
 "'annualCurrentDebt'": 86,
 "'annualInterestExpense'": 57,
 "'annualInterestExpenseNonOperating'": 57,
 "'annualCurrentAssets'": 55,
 "'annualTotalNonCurrentAssets'": 55,
 "'annualOperatingIncome'": 55,
 "'annualLongTermDebt'": 41,
 "'annualTotalOtherFinanceCost'": 27,
 "'annualReconciledDepreciation'": 25,
 "'annualGoodwillAndOtherIntangibleAssets'": 25,
 "'annualTotalDebt'": 16,
 "'annualNormalizedEBITDA'": 6}

In [22]:
missing_ratios_count = {}
for ratios in ticker_ratios.values():
    for ratio, value in ratios.items():
        if value is None:
            try:
                missing_ratios_count[ratio] += 1
            except KeyError:
                missing_ratios_count[ratio] = 1

missing_ratios_count = {k: missing_ratios_count[k] for k in sorted(missing_ratios_count.keys(), key=missing_ratios_count.get, reverse=True)}
missing_ratios_count

{'asset_coverage_ratio': 160,
 'quick_ratio': 138,
 'debt_service_coverage_ratio': 82,
 'cash_ratio': 56,
 'operating_cash_ratio': 56,
 'cash_coverage_ratio': 33,
 'interest_coverage_ratio': 29,
 'total_debt_to_EBITDA_ratio': 10,
 'total_debt_to_tangible_book_value_ratio': 7,
 'total_debt_to_total_assets_ratio': 7}

In [9]:
def pass_ratio(ticker, ratio, higher_better=True) -> bool:  # return whether pass or not
    assert ratio in ticker_ratios['MSFT'], "invalid ratio name!"  # just using msft as sample, no specific meaning
    ratio_value = ticker_ratios[ticker][ratio]
    ticker_sector = ticker_info.loc[ticker_info['Ticker'] == ticker]['GICS Sector'].to_list()[0]
    same_sector_tickers = ticker_info[ticker_info['GICS Sector'] == ticker_sector]['Ticker'].to_list()
    ratios = [ticker_ratios[ticker][ratio] for ticker in same_sector_tickers if ticker_ratios[ticker][ratio] is not None]  # TODO: see what to do to handle None values, for now just ignore
    ratios.append(ratio_value)
    ratios.sort()  # ascending
    percentile = ratios.index(ratio_value) / len(ratios) * 100  # top x percentile
    return percentile >= 50 if higher_better else percentile <= 50

def screen_stock(ticker: str) -> bool:
    ticker = ticker.upper()
    if not altman_z_score(ticker, latest=True, period=-1) > 1.1:
        return False

    # Ratios {ratio name: higher better}
    key_ratios = {'quick_ratio': True, 'debt_service_coverage_ratio': True, 'total_debt_to_total_assets_ratio': False, 'total_debt_to_EBITDA_ratio': False}  # must pass
    normal_ratios = {'cash_ratio': True, 'operating_cash_ratio': True, 'interest_coverage_ratio': True, 'asset_coverage_ratio': True, 'cash_coverage_ratio': True, 'total_debt_to_tangible_book_value_ratio': False}
    passed_ratio_count = 0
    liquidity = False
    coverage = False
    leverage = False
    liquidity_ratios = ['quick_ratio', 'cash_ratio', 'operating_cash_ratio']
    coverage_ratios = ['interest_coverage_ratio', 'debt_service_coverage_ratio', 'asset_coverage_ratio', 'cash_coverage_ratio']
    leverage_ratios = ['total_debt_to_tangible_book_value_ratio', 'total_debt_to_total_assets_ratio', 'total_debt_to_EBITDA_ratio']
    for ratio, higher_better in key_ratios.items():
        if pass_ratio(ticker, ratio, higher_better):
            passed_ratio_count += 1
            liquidity = ratio in liquidity_ratios if not liquidity else True
            coverage = ratio in coverage_ratios if not coverage else True
            leverage = ratio in leverage_ratios if not leverage else True
        else:
            return False  # must pass all key ratios
    for ratio, higher_better in normal_ratios.items():
        if pass_ratio(ticker, ratio, higher_better):
            passed_ratio_count += 1
            liquidity = ratio in liquidity_ratios if not liquidity else True
            coverage = ratio in coverage_ratios if not coverage else True
            leverage = ratio in leverage_ratios if not leverage else True
    if passed_ratio_count < 7 or not (liquidity and coverage and leverage):  # must pass 7/10 ratios and at least 1 in each category
        return False

    # F-score
    if F_score(ticker, latest=True, period=-1) < 6:  # F score need to be >= 6 to pass
        return False
    return True
F_score('msft', latest=True, period=-1)

'annualEBITDA'
'annualEBITDA'


5